<a href="https://colab.research.google.com/github/Sandeep354/Binary-Movie-Ratings-RBM/blob/main/Movie_Ratings_BM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Boltzmann Machine

##Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

##Downloading the dataset

1. Download the zip folder from online
2. Unzip the file you want
3. See that the unzipped file is them present in colab files 

###ML-1M

In [2]:
!wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip ml-1m.zip
!ls

--2020-10-28 05:39:04--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  19.0MB/s    in 0.3s    

2020-10-28 05:39:05 (19.0 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         
ml-1m  ml-1m.zip  sample_data


###ML-100K

In [3]:
!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls

--2020-10-28 05:39:05--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  16.1MB/s    in 0.3s    

2020-10-28 05:39:06 (16.1 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

## Importing the dataset


In [4]:
movies = pd.read_csv('ml-1m/movies.dat',
                     sep = '::', 
                     header = None, 
                     engine = 'python', 
                     encoding = 'latin-1')

In [5]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
users = pd.read_csv('ml-1m/users.dat',
                     sep = '::', 
                     header = None, 
                     engine = 'python', 
                     encoding = 'latin-1')

In [7]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [8]:
ratings = pd.read_csv('ml-1m/ratings.dat',
                     sep = '::', 
                     header = None, 
                     engine = 'python', 
                     encoding = 'latin-1')

In [9]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


## Preparing the training set and the test set


In [10]:
# We have 5 .base (training_set) and .test (testing_set) files

# Open and check .base (u1.base) file --> you will see columns seperated by TAB (4 space) and not comma
# so we need to specify that (use delimeter rather than sep)
training_set = pd.read_csv('ml-100k/u1.base', delimiter='\t')

In [11]:
training_set.head()

,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561


In [12]:
len(training_set)/len(ratings)

0.07998228370270613

In [13]:
# We have to convert DataFrame into an Array
training_set = np.array(training_set, dtype='int')
training_set

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

In [14]:
# Test Set
testing_set = pd.read_csv("ml-100k/u1.test", delimiter='\t')
testing_set = np.array(testing_set, dtype='int')

In [15]:
testing_set

array([[        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       ...,
       [      459,       934,         3, 879563639],
       [      460,        10,         3, 882912371],
       [      462,       682,         5, 886365231]])

## Getting the number of users and movies


In [16]:
nb_users = int(max(max(training_set[:,0]), max(testing_set[:,0])))
nb_users

943

In [17]:
nb_movies = int(max(max(training_set[:,1]), max(testing_set[:,1])))
nb_movies

1682

## Converting the data into an array with users in lines and movies in columns


In [18]:
def convert(data):
  new_data = [] # We'll make a list containing lists of ratings of users
  for id_users in range (1, nb_users+1):
    # Ratings of the user
    # first, we need to get all the movies he rated
    id_movies = data[:,1][data[:,0] == id_users]
    id_ratings = data[:,2][data[:,0] == id_users]
    # Now, we also need to fill the ratings which were not rated by the user (fill by zero)
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data

In [19]:
training_set = convert(training_set)
testing_set = convert(testing_set)

In [20]:
training_set[0][:10]

[0.0, 3.0, 4.0, 3.0, 3.0, 0.0, 4.0, 1.0, 5.0, 0.0]

In [21]:
len(training_set)

943

## Converting the data into Torch tensors


In [22]:
# We will create (convert) TorchTensor rather than using the regular NumpyArray
training_set = torch.FloatTensor(training_set)
testing_set = torch.FloatTensor(testing_set)

In [23]:
training_set

tensor([[0., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])

## Converting the ratings into binary ratings 1 (Liked) or 0 (Not Liked)


In [24]:
# Replace all 0 by -1 (since these were originally not present)
# Now we are converting to 0 and 1, so we dont to confuse with the existing 0 (it will now be -1)
training_set[training_set == 0] = -1

# 'or' operator not recognised --> so we have to seperately write the code
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

In [25]:
testing_set[testing_set == 0] = -1
testing_set[testing_set == 1] = 0
testing_set[testing_set == 2] = 0
testing_set[testing_set >= 3] = 1

## Creating the architecture of the Neural Network


In [31]:
# We will create PROBABILITY GRAPHICAL MODEL

class RBM():

  # nv - num of visible nodes
  # nh - num of hidden nodes
  def __init__(self, nv, nh):
    # Initialize weights 
    self.W = torch.randn(nh, nv) # this will take (batch_size=nh, bias=nv) 
    # Initialize bias for hidden nodes
    self.a = torch.randn(1, nh)
    # Initialize bias for visible nodes
    self.b = torch.randn(1, nv)

  # We will preform GIBBS SAMPLING (for contrastive gradient)
  # x --> P(h/v) : Probability of h given v
  def sample_h(self, x):
    wx = torch.mm(x, self.W.t()) # making product (.t() -transpose- to perform matrix mult correctly)
    activation = wx + self.a.expand_as(wx) # So that bias is added in each mini batch
    p_h_given_v = torch.sigmoid(activation)
    return p_h_given_v, torch.bernoulli(p_h_given_v)

  # y --> P(v/h) : Probability of v given h
  def sample_v(self, y):
    wy = torch.mm(y, self.W) # transpose not required here
    activation = wy + self.b.expand_as(wy) 
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h, torch.bernoulli(p_v_given_h)

  # Aproximating the RBM Log-Likelihood Gradient(maximize it --> minimize energy)
  # Contrastive Gradient (alternative to Stoch gradient)
  # (self, input vector(visible nodes at beginning), visible nodes appearing after k sampling, P(h=1/v0), P(h=1/vk))
  def train(self, v0, vk, ph0, phk):
    # The next codes can be found in RBM article / contrastive divergence
    self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
    self.b += torch.sum((v0-vk), 0) # Add 0 just to make tensor of 2-Dim
    self.a += torch.sum((ph0-phk), 0)


In [32]:
nv = len(training_set[0])
nh = 100 # This is TUNEABLE
batch_size = 100 # This is also Tuneable

In [33]:
rbm = RBM(nv, nh)

## Training the RBM


In [34]:
nb_epochs = 10

for epoch in range(1, nb_epochs + 1):
  # create Loss Function (and initialize it to 0)
  train_loss = 0
  # To normalize the loss, make a counter (s, make it a float)
  s = 0.
  
  # Go through users in loop in steps of 100 (the batch size)
  for id_user in range(0, nb_users - batch_size, batch_size):
    # Creating input for Gibbs sampling and initializing it
    vk = training_set[id_user:id_user+batch_size]
    # Target : ratings already rated (at beginning, input is same as target)
    v0 = training_set[id_user:id_user+batch_size]
    # Initial Probability (P(h=1/v0))
    # use x,_ to only get the first element of return (if there are multiple return elements)
    ph0,_ = rbm.sample_h(v0) 

    # for loop for k-steps contrastive divergence
    # Random Walk - k=10
    for k in range(10):
      _,hk = rbm.sample_h(vk)
      # Visible nodes after kth sampling :
      _,vk = rbm.sample_v(hk)
      # We dont want to update the -1 ratings (no ratings)
      # We need to freeze those ratings (v0<0 or v0==-1, use anytings)
      vk[v0<0] = v0[v0<0]
    
    # Finally before training, we need P(h=1/vk)
    phk,_ = rbm.sample_h(vk)
    rbm.train(v0, vk, ph0, phk)

    # with new weights, measure the new loss (only for the existing ratings --> v0 >= 0)
    train_loss += torch.mean(torch.abs(v0[v0>=0]-vk[v0>=0]))
    s += 1.
    # train_loss/s --> Normalized Train Loss
  
  print ('epoch: ' + str(epoch) + ' loss: ' + str(train_loss/s))

epoch: 1 loss: tensor(0.3585)
epoch: 2 loss: tensor(0.2489)
epoch: 3 loss: tensor(0.2534)
epoch: 4 loss: tensor(0.2486)
epoch: 5 loss: tensor(0.2479)
epoch: 6 loss: tensor(0.2496)
epoch: 7 loss: tensor(0.2496)
epoch: 8 loss: tensor(0.2488)
epoch: 9 loss: tensor(0.2476)
epoch: 10 loss: tensor(0.2450)


## Testing the RBM


In [40]:
test_loss = 0
s = 0.
# Replace all batch_size by 1
for id_user in range(nb_users):
  v = training_set[id_user:id_user+1] #v-input to make predictions on
  vt = testing_set[id_user:id_user+1] #vt-target

  # Blind Walk - k=1 (so just remove the for loop)
  
  if len(vt[vt>=0]) > 0:
    _,h = rbm.sample_h(v)
    _,v = rbm.sample_v(h)

    test_loss += torch.mean(torch.abs(vt[vt>=0]-v[vt>=0]))
    s += 1.

print ('Test Loss: ' + str(test_loss/s))

Test Loss: tensor(0.2670)
